## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-09-03-09-46 +0000,nypost,10 people charged for ‘organized attack’ on IC...,https://nypost.com/2025/07/08/us-news/10-peopl...
1,2025-07-09-03-08-53 +0000,nyt,U.S. Botched a Deal to Swap Venezuelans in El ...,https://www.nytimes.com/2025/07/08/world/ameri...
2,2025-07-09-02-57-54 +0000,nyt,Marco Rubio Impersonation Under State Dept. In...,https://www.nytimes.com/2025/07/08/us/politics...
3,2025-07-09-02-52-23 +0000,nypost,Rep. Ilhan Omar’s activist daughter unemployed...,https://nypost.com/2025/07/08/us-news/rep-ilha...
4,2025-07-09-02-43-47 +0000,nyt,Ukraine Turns to Fishing Nets to Catch Russian...,https://www.nytimes.com/2025/07/07/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2315/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
156,trump,22
46,new,7
163,tariffs,7
350,tariff,6
90,china,5


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
16,2025-07-09-01-11-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...,51
37,2025-07-08-21-38-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,50
72,2025-07-08-13-59-19 +0000,nyt,Trump’s New Trade Threats Set Off Global Scram...,https://www.nytimes.com/2025/07/08/business/ec...,47
64,2025-07-08-16-43-22 +0000,nypost,Trump rips Putin during cabinet meeting for ta...,https://nypost.com/2025/07/08/us-news/trump-ri...,44
48,2025-07-08-19-28-44 +0000,nypost,Zelensky’s right-hand man thanks Trump for rev...,https://nypost.com/2025/07/08/us-news/zelensky...,43


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
16,51,2025-07-09-01-11-00 +0000,wsj,President Trump decided to delay the implement...,https://www.wsj.com/economy/trade/trump-tariff...
18,25,2025-07-09-01-00-00 +0000,wsj,Two Republicans named Kevin are vying to be th...,https://www.wsj.com/economy/central-banking/tr...
12,24,2025-07-09-01-49-09 +0000,nypost,College group Zohran Mamdani co-founded welcom...,https://nypost.com/2025/07/08/us-news/college-...
24,24,2025-07-08-23-26-50 +0000,nypost,Mayor Eric Adams builds out re-election campai...,https://nypost.com/2025/07/08/us-news/mayor-er...
36,24,2025-07-08-21-43-53 +0000,latimes,L.A. business leaders express tariff frustrati...,https://www.latimes.com/business/story/2025-07...
31,22,2025-07-08-22-42-03 +0000,nypost,Gavin Newsom says he’s ‘spellbound’ by AME bis...,https://nypost.com/2025/07/08/us-news/gavin-ne...
48,21,2025-07-08-19-28-44 +0000,nypost,Zelensky’s right-hand man thanks Trump for rev...,https://nypost.com/2025/07/08/us-news/zelensky...
37,21,2025-07-08-21-38-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
3,19,2025-07-09-02-52-23 +0000,nypost,Rep. Ilhan Omar’s activist daughter unemployed...,https://nypost.com/2025/07/08/us-news/rep-ilha...
27,19,2025-07-08-23-10-28 +0000,nypost,"Texas’ early deployment of choppers, boats may...",https://nypost.com/2025/07/08/us-news/texas-ea...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
